## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df = application_df.drop(columns=["EIN","NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
application_types_to_replace = application_df["APPLICATION_TYPE"].value_counts().loc[lambda x : x < 200].keys().to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
application_df["CLASSIFICATION"].value_counts().loc[lambda x : x >1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace = application_df["CLASSIFICATION"].value_counts().loc[lambda x : x < 1000].keys().to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
encoded_df = pd.get_dummies(application_df)
encoded_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
X = encoded_df.drop("IS_SUCCESSFUL", axis=1)
y = encoded_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
len(X.columns)

43

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=50, input_dim=len(X.columns), activation="relu"))

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=50, activation="relu"))

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                2200      
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 4,801
Trainable params: 4,801
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

In [16]:
# Train the model
#  YOUR CODE GOES HERE

##checkpoint that saves model weights every 5 epochs
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', save_freq=5)

nn.fit(X_train_scaled, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(X_test_scaled, y_test), callbacks=[checkpoint])

Epoch 1/10
  1/201 [..............................] - ETA: 2:40 - loss: 0.7298 - accuracy: 0.4844
Epoch 1: loss improved from inf to 0.68975, saving model to best_model.hdf5

Epoch 1: loss improved from 0.68975 to 0.68043, saving model to best_model.hdf5

Epoch 1: loss improved from 0.68043 to 0.67155, saving model to best_model.hdf5
 15/201 [=>............................] - ETA: 0s - loss: 0.6716 - accuracy: 0.6010  
Epoch 1: loss improved from 0.67155 to 0.66453, saving model to best_model.hdf5

Epoch 1: loss improved from 0.66453 to 0.65663, saving model to best_model.hdf5

Epoch 1: loss improved from 0.65663 to 0.65007, saving model to best_model.hdf5
 31/201 [===>..........................] - ETA: 0s - loss: 0.6496 - accuracy: 0.6431
Epoch 1: loss improved from 0.65007 to 0.64288, saving model to best_model.hdf5

Epoch 1: loss improved from 0.64288 to 0.63816, saving model to best_model.hdf5

Epoch 1: loss improved from 0.63816 to 0.63357, saving model to best_model.hdf5
 45/201 

 62/201 [========>.....................] - ETA: 0s - loss: 0.5489 - accuracy: 0.7319
Epoch 3: loss did not improve from 0.54020

Epoch 3: loss did not improve from 0.54020

Epoch 3: loss did not improve from 0.54020

Epoch 3: loss did not improve from 0.54020

Epoch 3: loss did not improve from 0.54020
 83/201 [===========>..................] - ETA: 0s - loss: 0.5483 - accuracy: 0.7330
Epoch 3: loss did not improve from 0.54020

Epoch 3: loss did not improve from 0.54020

Epoch 3: loss did not improve from 0.54020

Epoch 3: loss did not improve from 0.54020
106/201 [==============>...............] - ETA: 0s - loss: 0.5481 - accuracy: 0.7329
Epoch 3: loss did not improve from 0.54020

Epoch 3: loss did not improve from 0.54020

Epoch 3: loss did not improve from 0.54020

Epoch 3: loss did not improve from 0.54020

Epoch 3: loss did not improve from 0.54020
128/201 [==================>...........] - ETA: 0s - loss: 0.5523 - accuracy: 0.7291
Epoch 3: loss did not improve from 0.54020

Epo


Epoch 6: loss did not improve from 0.49789

Epoch 6: loss did not improve from 0.49789

Epoch 6: loss did not improve from 0.49789
 65/201 [========>.....................] - ETA: 0s - loss: 0.5430 - accuracy: 0.7347
Epoch 6: loss did not improve from 0.49789

Epoch 6: loss did not improve from 0.49789

Epoch 6: loss did not improve from 0.49789

Epoch 6: loss did not improve from 0.49789
 88/201 [============>.................] - ETA: 0s - loss: 0.5456 - accuracy: 0.7315
Epoch 6: loss did not improve from 0.49789

Epoch 6: loss did not improve from 0.49789

Epoch 6: loss did not improve from 0.49789

Epoch 6: loss did not improve from 0.49789

Epoch 6: loss did not improve from 0.49789
110/201 [===============>..............] - ETA: 0s - loss: 0.5475 - accuracy: 0.7310
Epoch 6: loss did not improve from 0.49789

Epoch 6: loss did not improve from 0.49789

Epoch 6: loss did not improve from 0.49789

Epoch 6: loss did not improve from 0.49789
132/201 [==================>...........] - E


Epoch 9: loss did not improve from 0.49789

Epoch 9: loss did not improve from 0.49789
 67/201 [=========>....................] - ETA: 0s - loss: 0.5453 - accuracy: 0.7333
Epoch 9: loss did not improve from 0.49789

Epoch 9: loss did not improve from 0.49789

Epoch 9: loss did not improve from 0.49789

Epoch 9: loss did not improve from 0.49789
 88/201 [============>.................] - ETA: 0s - loss: 0.5476 - accuracy: 0.7316
Epoch 9: loss did not improve from 0.49789

Epoch 9: loss did not improve from 0.49789

Epoch 9: loss did not improve from 0.49789

Epoch 9: loss did not improve from 0.49789
110/201 [===============>..............] - ETA: 0s - loss: 0.5489 - accuracy: 0.7302
Epoch 9: loss did not improve from 0.49789

Epoch 9: loss did not improve from 0.49789

Epoch 9: loss did not improve from 0.49789

Epoch 9: loss did not improve from 0.49789

Epoch 9: loss did not improve from 0.49789
132/201 [==================>...........] - ETA: 0s - loss: 0.5487 - accuracy: 0.7297
Epo

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5547 - accuracy: 0.7282 - 301ms/epoch - 1ms/step
Loss: 0.5547195672988892, Accuracy: 0.7281632423400879
